In [1]:
import statsapi
import numpy as np

BATTER_STATS = [
    "gamesPlayed",
    "flyOuts",
    "groundOuts",
    "airOuts",
    "runs",
    "doubles",
    "triples",
    "homeRuns",
    "strikeOuts",
    "baseOnBalls",
    "intentionalWalks",
    "hits",
    "hitByPitch",
    "avg",
    "atBats",
    "obp",
    "slg",
    "ops",
    "caughtStealing",
    "stolenBases",
    "stolenBasePercentage",
    "groundIntoDoublePlay",
    "groundIntoTriplePlay",
    "numberOfPitches",
    "plateAppearances",
    "totalBases",
    "rbi",
    "leftOnBase",
    "sacBunts",
    "sacFlies",
    "babip",
    "groundOutsToAirouts",
    "catchersInterference",
    "atBatsPerHomeRun",
]

ZERO_VALUES = ['.---', '-.--']

def get_batter_stats(player_id):
    player = statsapi.player_stat_data(
        player_id, group="[hitting]", type="[season,lastXGames],limit=5"
    )
    stats = player["stats"]
    season = None
    last_x_batting = None
    for stat in stats:
        if stat["group"] == "hitting" and stat["type"] == "season":
            season = stat["stats"]
        elif stat["group"] == "hitting" and stat["type"] == "lastXGames":
            last_x_batting = stat["stats"]

    # Turn feature set into a numpy array
    features = []
    for val in BATTER_STATS:
        # First do the season stats:
        season_val = season.get(val, 0.0)
        if season_val in ZERO_VALUES:
            season_val = 0.0
        features.append(float(season_val))

    # Now do the last x games stats:
    for val in BATTER_STATS:
        last_x_val = last_x_batting.get(val, 0.0)
        if last_x_val in ZERO_VALUES:
            last_x_val = 0.0
        features.append(float(last_x_val))

    return features

In [ ]:
import statsapi
import numpy as np

PITCHER_STATS = ['gamesPlayed', 'gamesStarted', 'flyOuts', 'groundOuts', 'airOuts', 'runs', 'doubles', 'triples', 'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch', 'avg', 'atBats', 'obp', 'slg', 'ops', 'caughtStealing', 'stolenBases', 'stolenBasePercentage', 'groundIntoDoublePlay', 'numberOfPitches', 'era', 'inningsPitched', 'wins', 'losses', 'saves', 'saveOpportunities', 'holds', 'blownSaves', 'earnedRuns', 'whip', 'battersFaced', 'outs', 'gamesPitched', 'completeGames', 'shutouts', 'strikes', 'strikePercentage', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffs', 'totalBases', 'groundOutsToAirouts', 'winPercentage', 'pitchesPerInning', 'gamesFinished', 'strikeoutWalkRatio', 'strikeoutsPer9Inn', 'walksPer9Inn', 'hitsPer9Inn', 'runsScoredPer9', 'homeRunsPer9', 'inheritedRunners', 'inheritedRunnersScored', 'catchersInterference', 'sacBunts', 'sacFlies']

ZERO_VALUES = ['.---', '-.--']

def get_pitcher_stats(player_id):
    player = statsapi.player_stat_data(
        player_id, group="[pitching]", type="[season,lastXGames],limit=5"
    )
    stats = player["stats"]
    season = None
    season_stats = []
    last_x_pitching = None
    last_x_pitching_stats = []
    # Check if there's multiple season stats and lastXGames stats- these indicate a trade. We only care about current team
    # Index values are- 0=total, 1=new_team, 2=old_team
    for stat in stats:
        if stat["group"] == "pitching" and stat["type"] == "season":
            season_stats.append(stat["stats"])
        elif stat["group"] == "pitching" and stat["type"] == "lastXGames":
            last_x_pitching_stats.append(stat["stats"])
    if len(season_stats) > 1:
        season = season_stats[1]
    else:
        season = season_stats[0]

    if len(last_x_pitching_stats) > 1:
        last_x_pitching = last_x_pitching_stats[1]
    else:
        last_x_pitching = last_x_pitching_stats[0]

    # Turn feature set into a numpy array
    features = []
    for val in PITCHER_STATS:
        # First do the season stats:
        season_val = season.get(val, 0.0)
        if season_val in ZERO_VALUES:
            season_val = 0.0
        features.append(float(season_val))

    # Now do the last x games stats:
    for val in PITCHER_STATS:
        last_x_val = last_x_pitching.get(val, 0.0)
        if last_x_val in ZERO_VALUES:
            last_x_val = 0.0
        features.append(float(last_x_val))

    return features

kikuchi = 579328
assert len(get_pitcher_stats(kikuchi)) == len(PITCHER_STATS)*2 

In [ ]:
import statsapi

def get_lineup_player_stats(game_id):
    # Get game details
    game_response = statsapi.get("game", {"gamePk": game_id})

    # Extract lineup information
    away_lineup = game_response["liveData"]["boxscore"]["teams"]["away"]["battingOrder"]
    away_starting_pitcher_id = game_response["liveData"]["boxscore"]["teams"]["away"]["pitchers"][0] # Astros
    away_runs = game_response["liveData"]["linescore"]["teams"]["away"]["runs"]
    
    home_lineup = game_response["liveData"]["boxscore"]["teams"]["home"]["battingOrder"]
    home_starting_pitcher_id = game_response["liveData"]["boxscore"]["teams"]["home"]["pitchers"][0] # Orioles
    home_runs = game_response["liveData"]["linescore"]["teams"]["home"]["runs"]

    # Feature set: [[OPPONENT_STARTING_PITCHER],[ROSTER]]
    away_opponent_pitcher_stats = get_pitcher_stats(home_starting_pitcher_id)
    away_batting_stats = []
    for batter in away_lineup:
        away_batting_stats.extend(get_batter_stats(batter))

    home_opponent_pitcher_stats = get_pitcher_stats(away_starting_pitcher_id)
    home_batting_stats = []
    for batter in home_lineup:
        home_batting_stats.extend(get_batter_stats(batter))
    
    home_features = [] + home_batting_stats + home_opponent_pitcher_stats
    away_features = [] + away_batting_stats + away_opponent_pitcher_stats

    return (home_runs, home_features), (away_runs, away_features)


# Example usage
game_id = 746994  # Replace with your game ID
(home_runs, home_features), (away_runs, away_features) = get_lineup_player_stats(game_id)

In [3]:
import statsapi

years = ['2021', '2022', '2023']
validation_years = ['2024']

for year in years:
    games = statsapi.schedule(start_date=f"04/01/{year}", end_date=f"09/30/{year}")
    for game in games:
        game_id = game['game_id']
        (home_runs, home_features), (away_runs, away_features) = get_lineup_player_stats(game_id)
        print(home_runs, home_features, away_runs, away_features)

KeyboardInterrupt: 

In [ ]:
import statsapi


def get_player_stats(player_id):
    player = statsapi.player_stat_data(player_id, group="[last5Games]")
    return player


get_player_stats(592518)